In [3]:
!pip install --upgrade pip
# --- SETUP ---

!pip install openpyxl pandas numpy matplotlib seaborn --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- LOAD DATA ---
# (1) Đọc file Excel giá nguyên liệu
price_raw = pd.read_excel('gia_nguyen_lieu.xlsx')

# (2) Đọc dữ liệu báo cáo tài chính
steel_finance = pd.read_excel('bao_cao_tai_chinh1.xlsx')

file_nb = "Econ-index.ipynb"

with open(file_nb, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

file_gia_thep = "gia_lich_su_thep1.xlsx"
df_gia_thep = pd.read_excel(file_gia_thep)
print("Data from gia_lich_su_thep1.xlsx:")
display(df_gia_thep.head())










Data from gia_lich_su_thep1.xlsx:


,time,open,high,low,close,volume
0,2021-10-28,33.20,33.37,32.85,33.25,21923800
1,2021-10-29,33.25,33.31,32.74,32.74,23878000
2,2021-11-01,32.74,32.74,31.93,31.93,45218500
3,2021-11-02,31.82,32.34,31.65,31.82,25669500
4,2021-11-03,32.22,32.28,31.70,31.88,29867100


In [4]:
# Install libraries if needed
!pip install requests pandas --quiet

import requests
import pandas as pd
from functools import reduce

# ✅ Country - Only Vietnam
country = "VNM"

# ✅ Indicators required (Updated with IIP)
indicators = {
    "GDP Growth (%)": "NY.GDP.MKTP.KD.ZG",
    "CPI (Inflation, annual %)": "FP.CPI.TOTL.ZG",
    "Exchange Rate (USD/VND)": "PA.NUS.FCRF",
    "Interest Rate (%) (Lending Rate Proxy)": "FR.INR.LEND",
    "FDI Inflow (US$)": "BX.KLT.DINV.CD.WD",
    "Industrial Production Growth (IIP %)": "IP.PRD.INDX.ZG"
}

start_year = 2010
end_year = 2024

# ✅ World Bank API data fetch function
def get_wb_data(country_code, indicator_code, start, end):
    url = (
        f"https://api.worldbank.org/v2/country/{country_code}/indicator/"
        f"{indicator_code}?format=json&date={start}:{end}&per_page=2000"
    )
    r = requests.get(url)
    data = r.json()

    # Nếu không có dữ liệu
    if len(data) < 2 or not isinstance(data[1], list) or len(data[1]) == 0:
        print(f"⚠️ Warning: No data for indicator {indicator_code}")
        return pd.DataFrame()

    df = pd.DataFrame(data[1])[["date", "value"]]
    df["Year"] = df["date"].astype(int)
    df.drop(columns=["date"], inplace=True)
    df.rename(columns={"value": indicator_code}, inplace=True)
    return df

# ✅ Retrieve all indicators for Vietnam
data_frames = []
for name, code in indicators.items():
    print(f"Fetching {name} ...")
    df_temp = get_wb_data(country, code, start_year, end_year)
    if df_temp.empty:
        continue  # bỏ DataFrame rỗng
    df_temp.rename(columns={code: name}, inplace=True)
    data_frames.append(df_temp)

# ✅ Merge into a single table, chỉ khi có DataFrame tồn tại
if data_frames:
    df = reduce(lambda left, right: pd.merge(left, right, on="Year", how="outer"), data_frames)
    df = df.sort_values("Year").reset_index(drop=True)
else:
    df = pd.DataFrame()
    print("❌ No data available for any indicator!")

# ✅ Formatting display
pd.options.display.float_format = '{:,.2f}'.format

# ✅ Export Excel nếu có dữ liệu
if not df.empty:
    output_filename = "Vietnam_Macro_Indicators_2010_2024.xlsx"
    df.to_excel(output_filename, index=False)
    print("✅ File exported successfully:", output_filename)
else:
    print("❌ No Excel file exported, data empty.")

df

Fetching GDP Growth (%) ...
Fetching CPI (Inflation, annual %) ...
Fetching Exchange Rate (USD/VND) ...
Fetching Interest Rate (%) (Lending Rate Proxy) ...
Fetching FDI Inflow (US$) ...
Fetching Industrial Production Growth (IIP %) ...
⚠️ Warning: No data for indicator IP.PRD.INDX.ZG
✅ File exported successfully: Vietnam_Macro_Indicators_2010_2024.xlsx


,GDP Growth (%),Year,"CPI (Inflation, annual %)",Exchange Rate (USD/VND),Interest Rate (%) (Lending Rate Proxy),FDI Inflow (US$)
0,6.42,2010,9.21,"18,612.92",13.14,8000000000
1,6.41,2011,18.68,"20,509.75",16.95,7430000000
2,5.50,2012,9.09,"20,828.00",13.47,8368000000
3,5.55,2013,6.59,"20,933.42",10.37,8900000000
4,6.42,2014,4.08,"21,148.00",8.66,9200000000
5,6.99,2015,0.63,"21,697.57",7.12,11800000000
6,6.69,2016,2.67,"21,935.00",6.96,12600000000
7,6.94,2017,3.52,"22,370.09",7.07,14100000000
8,7.47,2018,3.54,"22,602.05",7.37,15500000000
9,7.36,2019,2.80,"23,050.24",7.71,16120000000


In [5]:
import nbformat

file_nb = "Econ-index.ipynb"

# --- Đọc notebook ---
with open(file_nb, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)  # <-- đây là khối nằm trong 'with'

# Tiếp tục các bước khác, không cần thụt vào nữa
file_gia_thep = "gia_lich_su_thep1.xlsx"
df_gia_thep = pd.read_excel(file_gia_thep)

file_econ =  'Vietnam_Macro_Indicators_2010_2024.xlsx'
df_econ = pd.read_excel(file_econ)

print("Data from gia_lich_su_thep1.xlsx:")
display(df_gia_thep.head())
display(price_raw.head())
display(steel_finance.head())
display(df_econ.head())

Data from gia_lich_su_thep1.xlsx:


,time,open,high,low,close,volume
0,2021-10-28,33.20,33.37,32.85,33.25,21923800
1,2021-10-29,33.25,33.31,32.74,32.74,23878000
2,2021-11-01,32.74,32.74,31.93,31.93,45218500
3,2021-11-02,31.82,32.34,31.65,31.82,25669500
4,2021-11-03,32.22,32.28,31.70,31.88,29867100


,Ngày,Đơn vị: USD,% Thay đổi,Lần cuối,1,USD,=,"26,301.50",VND
0,2025-11-01,103.34,-0.02,"2,717,997.01",vnd,/tấn,NaN,NaN,NaN
1,2025-10-01,105.83,0.01,"2,783,487.75",vnd,/tấn,NaN,NaN,NaN
2,2025-09-01,105.29,0.03,"2,769,284.94",vnd,/tấn,NaN,NaN,NaN
3,2025-08-01,101.81,0.03,"2,677,755.71",vnd,/tấn,NaN,NaN,NaN
4,2025-07-01,99.12,0.05,"2,607,004.68",vnd,/tấn,NaN,NaN,NaN


,Unnamed: 0,CP,Năm,Kỳ,TÀI SẢN NGẮN HẠN (đồng),Tiền và tương đương tiền (đồng),Giá trị thuần đầu tư ngắn hạn (đồng),Các khoản phải thu ngắn hạn (đồng),Hàng tồn kho ròng,Tài sản lưu động khác,...,Vay và nợ thuê tài chính dài hạn (đồng),Người mua trả tiền trước ngắn hạn (đồng),Vay và nợ thuê tài chính ngắn hạn (đồng),Lợi thế thương mại (đồng),Trả trước dài hạn (đồng),Tài sản dài hạn khác (đồng),Phải thu dài hạn khác (đồng),Phải thu dài hạn (đồng),symbol,Các quỹ khác
0,0,HPG,2025,3,95425988115582,9092562739415,"18,903,949,054,642.00",13727194725111,45623656431728,8078625164686,...,28356095137594,619205121131,68482383626253,"52,097,751,283.00",4390536576181,0.00,825562606009,849788287676,HPG,NaN
1,1,HPG,2025,2,97613904938535,10688024277258,"17,584,027,436,423.00",12334925649801,48853408114115,8153519460938,...,27312288660990,306667783670,66692028477152,"57,563,156,071.00",4338725174159,0.00,827053815447,900453913211,HPG,NaN
2,2,HPG,2025,1,88914195573724,7459035818129,"16,212,510,112,252.00",10856053423195,46603844415849,7782751804299,...,27256965869238,570631436241,61784512821075,"60,588,141,397.00",4261916994097,0.00,848015622228,920743791614,HPG,NaN
3,3,HPG,2024,4,86674276272995,6887646139852,"18,974,716,730,905.00",7647800286988,46091222189472,7072890925778,...,27080443256096,739178306553,55882686213459,"63,613,126,723.00",4269141694868,0.00,840594835822,923400123614,HPG,NaN
4,4,HPG,2024,3,80841803394106,8500998079280,"16,386,803,148,945.00",9140948029240,40197821463879,6615232672762,...,24517626484538,440576623880,54180722680952,"66,834,062,714.00",4347213561931,0.00,841682562478,1031145036427,HPG,NaN


,GDP Growth (%),Year,"CPI (Inflation, annual %)",Exchange Rate (USD/VND),Interest Rate (%) (Lending Rate Proxy),FDI Inflow (US$)
0,6.42,2010,9.21,"18,612.92",13.14,8000000000
1,6.41,2011,18.68,"20,509.75",16.95,7430000000
2,5.50,2012,9.09,"20,828.00",13.47,8368000000
3,5.55,2013,6.59,"20,933.42",10.37,8900000000
4,6.42,2014,4.08,"21,148.00",8.66,9200000000


In [6]:
print("=== 📊 File: gia_lich_su_thep1.xlsx ===")
print(df_gia_thep.columns.tolist())

print("\n=== 🧱 File: gia_nguyen_lieu.xlsx ===")
print(price_raw.columns.tolist())

print("\n=== 💰 File: bao_cao_tai_chinh1.xlsx ===")
print(steel_finance.columns.tolist())

print("\n=== 🌏 File: Vietnam_Macro_Indicators_2010_2024.xlsx ===")
print(df_econ.columns.tolist())

=== 📊 File: gia_lich_su_thep1.xlsx ===
['time', 'open', 'high', 'low', 'close', 'volume']

=== 🧱 File: gia_nguyen_lieu.xlsx ===
['Ngày', 'Đơn vị: USD', '% Thay đổi', 'Lần cuối', 1, 'USD', '=', 26301.5, 'VND']

=== 💰 File: bao_cao_tai_chinh1.xlsx ===
['Unnamed: 0', 'CP', 'Năm', 'Kỳ', 'TÀI SẢN NGẮN HẠN (đồng)', 'Tiền và tương đương tiền (đồng)', 'Giá trị thuần đầu tư ngắn hạn (đồng)', 'Các khoản phải thu ngắn hạn (đồng)', 'Hàng tồn kho ròng', 'Tài sản lưu động khác', 'TÀI SẢN DÀI HẠN (đồng)', 'Phải thu về cho vay dài hạn (đồng)', 'Tài sản cố định (đồng)', 'Giá trị ròng tài sản đầu tư', 'Đầu tư dài hạn (đồng)', 'Lợi thế thương mại', 'Tài sản dài hạn khác', 'TỔNG CỘNG TÀI SẢN (đồng)', 'NỢ PHẢI TRẢ (đồng)', 'Nợ ngắn hạn (đồng)', 'Nợ dài hạn (đồng)', 'VỐN CHỦ SỞ HỮU (đồng)', 'Vốn và các quỹ (đồng)', 'Lãi chưa phân phối (đồng)', 'Vốn Ngân sách nhà nước và quỹ khác', 'LỢI ÍCH CỦA CỔ ĐÔNG THIỂU SỐ', 'TỔNG CỘNG NGUỒN VỐN (đồng)', 'Trả trước cho người bán ngắn hạn (đồng)', 'Phải thu về cho vay ng

In [7]:
# ============================================================
# 🧹 STEP 1: Chuẩn hóa giá cổ phiếu ngành thép (daily)
# ============================================================
df_gia_thep['time'] = pd.to_datetime(df_gia_thep['time'])
df_gia_thep = df_gia_thep.sort_values('time')
df_gia_thep.rename(columns={'time': 'Date', 'close': 'Stock_Close'}, inplace=True)

# ============================================================
# ⚙️ STEP 2: Chuẩn hóa dữ liệu giá nguyên liệu (monthly)
# ============================================================
# Giả định 'Ngày' là dạng chuỗi ngày đầu tháng hoặc cuối tháng
price_raw.rename(columns={'Ngày': 'Date'}, inplace=True)
price_raw['Date'] = pd.to_datetime(price_raw['Date'], errors='coerce')
price_raw = price_raw[['Date', 'Lần cuối']].rename(columns={'Lần cuối': 'Input_Price'})
price_raw = price_raw.dropna(subset=['Date'])

# ============================================================
# 💰 STEP 3: Chuẩn hóa báo cáo tài chính (quarterly)
# ============================================================
finance = steel_finance.copy()
finance = finance.rename(columns={'CP': 'Symbol', 'Năm': 'Year', 'Kỳ': 'Quarter'})
# Chuyển 'Năm' và 'Kỳ' thành cột thời gian thực
finance['Quarter'] = finance['Quarter'].astype(str).str.replace('Q', '')
finance['Quarter'] = pd.to_numeric(finance['Quarter'], errors='coerce')
finance['Date'] = pd.to_datetime(finance['Year'].astype(str) + '-' + (finance['Quarter']*3).astype(int).astype(str) + '-01', errors='coerce')
finance = finance.dropna(subset=['Date'])
finance = finance[['Symbol', 'Date', 'TỔNG CỘNG TÀI SẢN (đồng)', 'VỐN CHỦ SỞ HỮU (đồng)', 'NỢ PHẢI TRẢ (đồng)']]

# ============================================================
# 🌏 STEP 4: Chuẩn hóa chỉ số kinh tế vĩ mô (annual)
# ============================================================
econ = df_econ.copy()
econ['Year'] = econ['Year'].astype(int)
econ['Date'] = pd.to_datetime(econ['Year'].astype(str) + '-12-31')
econ = econ[['Date', 'GDP Growth (%)', 'CPI (Inflation, annual %)', 
             'Exchange Rate (USD/VND)', 'Interest Rate (%) (Lending Rate Proxy)', 
             'FDI Inflow (US$)']]

# ============================================================
# 🔗 STEP 5: Merge dữ liệu
# ============================================================

# Merge (outer để không mất dữ liệu)
df_merged = pd.merge_asof(
    df_gia_thep.sort_values('Date'),
    price_raw.sort_values('Date'),
    on='Date',
    direction='backward'
)

df_merged = pd.merge_asof(
    df_merged.sort_values('Date'),
    finance.sort_values('Date'),
    on='Date',
    direction='backward'
)

df_merged = pd.merge_asof(
    df_merged.sort_values('Date'),
    econ.sort_values('Date'),
    on='Date',
    direction='backward'
)

# ============================================================
# 📦 STEP 6: Xuất dữ liệu tổng hợp
# ============================================================
output_file = "steel_data_ready_for_ML.xlsx"
df_merged.to_excel(output_file, index=False)

print("✅ Data merged successfully!")
print(f"📁 Saved as: {output_file}")
display(df_merged.head(10))

✅ Data merged successfully!
📁 Saved as: steel_data_ready_for_ML.xlsx


,Date,open,high,low,Stock_Close,volume,Input_Price,Symbol,TỔNG CỘNG TÀI SẢN (đồng),VỐN CHỦ SỞ HỮU (đồng),NỢ PHẢI TRẢ (đồng),GDP Growth (%),"CPI (Inflation, annual %)",Exchange Rate (USD/VND),Interest Rate (%) (Lending Rate Proxy),FDI Inflow (US$)
0,2021-10-28,33.20,33.37,32.85,33.25,21923800,"3,188,530.85",TVN,26663482146611,11058977702940,15604504443671,2.87,3.22,"23,208.37",7.65,15800000000
1,2021-10-29,33.25,33.31,32.74,32.74,23878000,"3,188,530.85",TVN,26663482146611,11058977702940,15604504443671,2.87,3.22,"23,208.37",7.65,15800000000
2,2021-11-01,32.74,32.74,31.93,31.93,45218500,"2,497,853.46",TVN,26663482146611,11058977702940,15604504443671,2.87,3.22,"23,208.37",7.65,15800000000
3,2021-11-02,31.82,32.34,31.65,31.82,25669500,"2,497,853.46",TVN,26663482146611,11058977702940,15604504443671,2.87,3.22,"23,208.37",7.65,15800000000
4,2021-11-03,32.22,32.28,31.70,31.88,29867100,"2,497,853.46",TVN,26663482146611,11058977702940,15604504443671,2.87,3.22,"23,208.37",7.65,15800000000
5,2021-11-04,31.88,32.62,31.88,32.28,14849100,"2,497,853.46",TVN,26663482146611,11058977702940,15604504443671,2.87,3.22,"23,208.37",7.65,15800000000
6,2021-11-05,32.28,32.68,31.93,32.45,19377600,"2,497,853.46",TVN,26663482146611,11058977702940,15604504443671,2.87,3.22,"23,208.37",7.65,15800000000
7,2021-11-08,32.62,32.68,32.22,32.56,21255400,"2,497,853.46",TVN,26663482146611,11058977702940,15604504443671,2.87,3.22,"23,208.37",7.65,15800000000
8,2021-11-09,32.56,33.02,32.34,32.39,24717300,"2,497,853.46",TVN,26663482146611,11058977702940,15604504443671,2.87,3.22,"23,208.37",7.65,15800000000
9,2021-11-10,32.39,32.45,31.65,31.70,43376600,"2,497,853.46",TVN,26663482146611,11058977702940,15604504443671,2.87,3.22,"23,208.37",7.65,15800000000
